# Dependencies

In [4]:
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/liudvikas/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
model = YOLO('yolov8s-worldv2.pt')  # You can use 'yolov8.pt' or another YOLO version

100%|██████████| 24.7M/24.7M [00:05<00:00, 4.93MB/s]


# Input

In [14]:
from huggingface_hub import hf_hub_download
for i in range(1):
    hf_hub_download(repo_id="osv5m/osv5m", filename=str(i).zfill(2)+'.zip', subfolder="images/train", repo_type='dataset', local_dir="datasets/OpenWorld")
    hf_hub_download(repo_id="osv5m/osv5m", filename="README.md", repo_type='dataset', local_dir="datasets/OpenWorld")
hf_hub_download(repo_id="osv5m/osv5m", filename="train.csv", repo_type='dataset', local_dir="datasets/OpenWorld")

00.zip:   0%|          | 0.00/2.52G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [8]:
import os
import zipfile
for root, dirs, files in os.walk("datasets"):
    for file in files:
        if file.endswith(".zip"):
            with zipfile.ZipFile(os.path.join(root, file), 'r') as zip_ref:
                zip_ref.extractall(root)
                os.remove(os.path.join(root, file))

In [10]:
image_path = '/kaggle/input/test-image/test.jpeg'  # Replace with your image path

custom_classes = ['blue post box']

In [ ]:
import pandas as pd
train_df = pd.read_csv('datasets/OpenWorld/train.csv')

/var/folders/8n/7kcjbmm92wv4f2ldlmlw82r40000gn/T/ipykernel_17965/2763937421.py:2: DtypeWarning: Columns (11,27) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('datasets/OpenWorld/train.csv')


In [9]:
sf_df = train_df[train_df['sub-region'] == 'San Francisco County']

In [11]:
sf_df.head()

,id,latitude,longitude,thumb_original_url,country,sequence,captured_at,lon_bin,lat_bin,cell,...,quadtree_10_50000,quadtree_10_12500,quadtree_10_500,quadtree_10_2500,unique_region,unique_sub-region,unique_city,unique_country,creator_username,creator_id
1088205,3605315972907949,37.788225,-122.431472,https://scontent-cdg4-3.xx.fbcdn.net/m1/v/t6/A...,US,dnlVvUc7OikdFHEIVTR9XQ,1473674665000,15,68,"(15, 68)",...,68,269,4035,1319,California_US,San Francisco County_California_US,San Francisco_San Francisco County_California_US,US,bhousel,1.073686e+14
1088207,4202157466516125,37.789148,-122.437256,https://scontent-cdg4-1.xx.fbcdn.net/m1/v/t6/A...,US,a8dxtjth9b5vyevo4l0n1c,1475057013000,15,68,"(15, 68)",...,68,269,4035,1319,California_US,San Francisco County_California_US,San Francisco_San Francisco County_California_US,US,bhousel,1.073686e+14
1088208,463618974874147,37.768582,-122.407943,https://scontent-cdg4-1.xx.fbcdn.net/m1/v/t6/A...,US,0WDeYUe1JonP8Hr9VKSq0A,1393200125000,15,68,"(15, 68)",...,68,269,4035,1319,California_US,San Francisco County_California_US,San Francisco_San Francisco County_California_US,US,jesolem,1.094448e+14
1088209,447801236318037,37.745189,-122.442009,https://scontent-cdg4-3.xx.fbcdn.net/m1/v/t6/A...,US,HGOhTj2vRZaSAYthOW3Q-g,1475229257000,15,68,"(15, 68)",...,68,269,4035,1319,California_US,San Francisco County_California_US,San Francisco_San Francisco County_California_US,US,bhousel,1.073686e+14
1088211,1655231981340841,37.758483,-122.425774,https://scontent-cdg4-3.xx.fbcdn.net/m1/v/t6/A...,US,CUAUBqnjoLihot2FbIYlFg,1424170691000,15,68,"(15, 68)",...,68,269,4035,1319,California_US,San Francisco County_California_US,San Francisco_San Francisco County_California_US,US,sarawabreus,1.107715e+14


### Load amenities from certain region

In [10]:
import osmnx as ox

# Define the bounding box or location
place_name = "San Francisco, California, USA"  # Example area
tags = {'amenity': ['post_box', 'bicycle_parking', 'bench', 'fast_food', 'waste_basket']}  # Fetch all amenities

# Fetch amenities in the defined area
gdf = ox.features_from_place(place_name, tags)

# Extract latitude and longitude from the geometries
gdf['latitude'] = gdf.geometry.centroid.y
gdf['longitude'] = gdf.geometry.centroid.x

# Filter for relevant columns
amenities_lat_lon_df = gdf[['amenity', 'latitude', 'longitude']]

# Display the DataFrame
print(amenities_lat_lon_df)

                           amenity   latitude   longitude
element_type osmid                                       
node         61680526     post_box  37.770046 -122.448534
             265689505    post_box  37.749487 -122.408708
             266903722    post_box  37.763169 -122.410314
             266903723    post_box  37.754490 -122.401539
             266903734    post_box  37.760782 -122.404006
...                            ...        ...         ...
way          1213210774  fast_food  37.788391 -122.397967
             1293537276      bench  37.807658 -122.472643
             1324192089      bench  37.783079 -122.478253
             1326151484      bench  37.735297 -122.376198
             1326151485      bench  37.735397 -122.376233

[6861 rows x 3 columns]


/var/folders/8n/7kcjbmm92wv4f2ldlmlw82r40000gn/T/ipykernel_17965/2471949541.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['latitude'] = gdf.geometry.centroid.y
/var/folders/8n/7kcjbmm92wv4f2ldlmlw82r40000gn/T/ipykernel_17965/2471949541.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['longitude'] = gdf.geometry.centroid.x


In [ ]:
import pandas as pd
from geopy.distance import geodesic

# Applying to sf_df
result_list = []
for _, row in sf_df.iterrows():
    picture_coords = (row['latitude'], row['longitude'])
    amenities_in_radius = amenities_lat_lon_df[
        amenities_lat_lon_df.apply(
            lambda x: geodesic(picture_coords, (x['latitude'], x['longitude'])).meters <= 30,
            axis=1
        )
    ]
    result_list.append({
        'latitude': row['latitude'],
        'longitude': row['longitude'],
        'id': row['id'],
        'amenities': amenities_in_radius.to_dict('records')
    })

# Create a new DataFrame from the results
final_df = pd.DataFrame(result_list)

# Display the resulting DataFrame
print(final_df)

      latitude   longitude                id  \
0    37.788225 -122.431472  3605315972907949   
1    37.789148 -122.437256  4202157466516125   
2    37.768582 -122.407943   463618974874147   
3    37.745189 -122.442009   447801236318037   
4    37.758483 -122.425774  1655231981340841   
..         ...         ...               ...   
552  37.806461 -122.404956   261388429046250   
553  37.738176 -122.412780   325898852284739   
554  37.797334 -122.426421   485611852584494   
555  37.809544 -122.416815   487553699130983   
556  37.771431 -122.430322  1005373900278738   

                                             amenities  
0                                                   []  
1                                                   []  
2    [{'amenity': 'post_box', 'latitude': 37.768397...  
3                                                   []  
4    [{'amenity': 'waste_basket', 'latitude': 37.75...  
..                                                 ...  
552  [{'amenity': 'waste

In [ ]:
# Filter out entries with no amenities
amenity_df = final_df[final_df['amenities'].apply(len) > 0]
amenity_df = amenity_df.reset_index(drop=True)

      latitude   longitude                id  \
0    37.768582 -122.407943   463618974874147   
1    37.758483 -122.425774  1655231981340841   
2    37.763731 -122.415169   304576254613199   
3    37.769407 -122.429197   813289055956371   
4    37.807794 -122.408051  2888091698095696   
..         ...         ...               ...   
106  37.767121 -122.461363  4296009723743687   
107  37.797085 -122.399549   242578727606559   
108  37.780785 -122.466423   519651605713361   
109  37.806461 -122.404956   261388429046250   
110  37.771431 -122.430322  1005373900278738   

                                             amenities  
0    [{'amenity': 'post_box', 'latitude': 37.768397...  
1    [{'amenity': 'waste_basket', 'latitude': 37.75...  
2    [{'amenity': 'post_box', 'latitude': 37.763584...  
3    [{'amenity': 'bench', 'latitude': 37.769538, '...  
4    [{'amenity': 'waste_basket', 'latitude': 37.80...  
..                                                 ...  
106  [{'amenity': 'bench

# Model

In [16]:
custom_classes = ["blue post box"]
model.set_classes(custom_classes)

# Inference

In [12]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [17]:
# Perform inference
results = model("test.jpeg", conf=0.01)

detections = []
for result in results:
    for box, cls, conf in zip(result.boxes.xyxy, result.boxes.cls, result.boxes.conf):
        # Convert class index to a string label
        detected_class = model.names[int(cls)]
        if detected_class in custom_classes:
            detection = {
                "class": detected_class,
                "confidence": float(conf)
            }
            detections.append(detection)

print(detections)


image 1/1 /Users/liudvikas/Programming/distance-vlm/test.jpeg: 448x640 1 blue post box, 133.7ms
Speed: 5.3ms preprocess, 133.7ms inference, 4.0ms postprocess per image at shape (1, 3, 448, 640)
[{'class': 'blue post box', 'confidence': 0.08160720020532608}]
